In [1]:
import requests
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2
import typing

from configparser import ConfigParser
from datetime import datetime
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2.extras import execute_values
from sqlalchemy import create_engine


from database_functions import get_config, cryptodb_connection

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#GLOBALS
CONFIG_FILEPATH="database.ini"
SECTION_DB_DEFAULT="postgres"
SECTION_DB_CREATE="crypto"
SECTION_API="api"

TABLE_NAME_TARGET='crypto_timeseries'

In [3]:
config_default = get_config(filename=CONFIG_FILEPATH, section=SECTION_DB_DEFAULT)
config_create = get_config(filename=CONFIG_FILEPATH, section=SECTION_DB_CREATE)
config_default
config_create


{'host': 'localhost',
 'database': 'postgres',
 'user': 'postgres',
 'password': 'postgres',
 'port': '5432'}

{'host': 'localhost',
 'database': 'crypto',
 'user': 'postgres',
 'password': 'postgres',
 'port': '5432'}

In [4]:
#create Database
con = psycopg2.connect(**config_default)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Obtain a DB Cursor
cursor          = con.cursor()

# Create database
try:
    cursor.execute(f"create database {config_create['database']};")
except Exception as e:
    print(f"""
                Following exception occurred: 
                    {e}
        """)


                Following exception occurred: 
                    database "crypto" already exists

        


In [5]:


#Create TABLES
with cryptodb_connection() as db:
        with db.conn.cursor() as curs:
            
            #create crypto_timeseries table
            curs.execute(
                            """ 
                                CREATE TABLE IF NOT EXISTS crypto_timeseries
                                (   
                                    index               SERIAL NOT NULL,
                                    timestamp           TIMESTAMP NOT NULL,
                                    id                  VARCHAR NOT NULL,
                                    rank                INT NOT NUll,
                                    symbol              VARCHAR NOT NULL,
                                    name                VARCHAR NOT NULL,
                                    supply              FLOAT,
                                    maxsupply           FLOAT,
                                    marketcapusd        FLOAT,
                                    volumeusd24hr       FLOAT,
                                    priceusd            FLOAT,
                                    changepercent24hr   FLOAT,
                                    vwap24hr            FLOAT,
                                    explorer            VARCHAR
                                );	
                            """
                        )
            
            #fecth tables in database
            curs.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
            print(curs.fetchall())

[('crypto_timeseries',)]
